# CASA0007 QUANTITATIVE METHODS - COURSEWORK 3

Title: How does Urban Greenery Affect Personal Well-Being in London?

Abstract: This paper investigates the effect of urban greenery on personal well-being in London and the associated pathway(s) by answering the following research questions:
- Does urban greenery affect personal well-being, as measured by ONS4?
- What is the pathway(s) by which urban greenery affect personal well-being?

In [1]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from math import log, sqrt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA 
from scipy.stats import norm

## Presentation of Data

### Personal Well-Being

In [2]:
# Read well-being
# Source: https://data.london.gov.uk/dataset/subjective-personal-well-being-borough
url = 'https://github.com/ngliangwei15/CASA0007-CW3/blob/main/Data/personal-well-being-borough.xlsx?raw=true'
wellbeing = pd.read_excel(url, sheet_name = 'Summary - Mean Scores', skiprows = 1)

# Rename and keep useful columns
wellbeing_cols = ['Life Satisfaction', 'Worthwhile', 'Happiness']
year = '2011/12'
wellbeing.rename({year: wellbeing_cols[0],
                 year+'.1': wellbeing_cols[1],
                 year+'.2': wellbeing_cols[2],
                 year+'.3': 'Anxiety'},
                axis = 1, inplace = True)
wellbeing = wellbeing[['Code', 'Area'] + wellbeing_cols + ['Anxiety']]

# Keep only data on London boroughs (Area Code starts with E09)
wellbeing.dropna(axis = 0, inplace = True)
wellbeing.drop(wellbeing[wellbeing.Area == "City of London"].index, inplace = True)
wellbeing = wellbeing[wellbeing['Code'].str.match(r'E09')].sort_values(by = ['Code'], ascending=True)
wellbeing = wellbeing.reset_index(drop = True)
wellbeing.drop('Code', axis = 1, inplace = True)

# Set data type
wellbeing[wellbeing_cols + ['Anxiety']] = wellbeing[wellbeing_cols + ['Anxiety']].astype('float', copy = False)

### Urban Greenery

In [3]:
# Read NDVI
# Source: https://data.london.gov.uk/dataset/green-and-blue-cover
url = 'https://github.com/ngliangwei15/CASA0007-CW3/blob/main/Data/green_cover_borough_summary_0.05.xlsx?raw=true'
ndvi_cols = ['lb_code', 'lb_name', 'percent_green']
ndvi = pd.read_excel(url, sheet_name = 'borough_green_cover', usecols = ndvi_cols)

# Drop City of London
ndvi.drop(ndvi[ndvi.lb_code == 'City of London'].index, inplace = True)
ndvi.sort_values(by = ['lb_name'], ascending=True)
ndvi = ndvi.reset_index(drop = True)

# Change name of 'City of Westminster' to Westminster
ndvi.loc[ndvi[ndvi.lb_code == 'City of Westminster'].index, 'lb_code'] = 'Westminster'

# Rename column
ndvi.drop('lb_name', axis = 1, inplace = True)
ndvi.rename({'lb_code': 'Area', 
             'percent_green': 'percentage_greenery'}, axis = 1, inplace = True)

### Mediators

#### Intrinsic well-being enhancement quality of greenery

In [4]:
anxiety = wellbeing.drop(wellbeing_cols, axis = 1)
anxiety.Anxiety = -anxiety.Anxiety
wellbeing.drop('Anxiety', axis = 1, inplace = True)

#### Promotion of physical activities

In [5]:
# Read in physical exercise
# Source: https://data.londonsport.org/dataset/e53dn/physical-activity-levels-by-borough
url = 'https://github.com/ngliangwei15/CASA0007-CW3/blob/main/Data/LSD21-APS-Physical-Activity-Levels.xlsx?raw=true'
exercise = pd.read_excel(url, sheet_name = '2012', usecols = ['GSS_Code', 'Region', 'Active'])

# Keep only data on London boroughs (Area Code starts with E09)
exercise = exercise[exercise['GSS_Code'].str.match(r'E09')].sort_values(by = ['GSS_Code'], ascending=True)
exercise = exercise.reset_index(drop = True)
exercise.drop('GSS_Code', axis = 1, inplace = True)

# Convert 'Active' to percentage
exercise.Active = exercise.Active*100

# Rename columns
exercise.rename({'Region': 'Area'}, axis = 1, inplace = True)

#### Enhancement of air quality

In [6]:
# Read in air quality 
# Source: https://data.london.gov.uk/dataset/llaqm-bespoke-borough-by-borough-air-quality-modelling-and-data
url = 'https://github.com/ngliangwei15/CASA0007-CW3/blob/main/Data/NO2_AnnualMean_2013_PopulationData.xlsx?raw=true'
air_quality = pd.read_excel(url, sheet_name = 'Borough exceeding 40', skiprows = 4, 
                            usecols=['Borough Name', 'PopExc2013 %'])
air_quality['Borough Name'] = air_quality['Borough Name'].str.replace('&', 'and')

# Drop City of London
air_quality.drop(air_quality[air_quality['Borough Name'] == 'City of London'].index, inplace = True)
air_quality = air_quality.reset_index(drop = True)

# Log
air_quality['air_quality'] = -air_quality['PopExc2013 %'].apply(log)
air_quality.drop('PopExc2013 %', axis = 1, inplace = True)

# Rename columns
air_quality.rename({'Borough Name': 'Area'},
                  axis = 1, inplace = True)

### Control Variables

#### Self-reported health and age

In [7]:
# Read self-reported health and age profile 
# Source: https://data.london.gov.uk/dataset/msoa-atlas
url = 'https://raw.githubusercontent.com/ngliangwei15/CASA0007-CW3/main/Data/msoa-data.csv'
age_cols = ['Age Structure (2011 Census);0-15;',
            'Age Structure (2011 Census);16-29;',
            'Age Structure (2011 Census);30-44;',
            'Age Structure (2011 Census);45-64;',
            'Age Structure (2011 Census);65+;']
health_cols = ['Health (2011 Census);Very good health;',
              'Health (2011 Census);Good health;',
              'Health (2011 Census);Fair health;',
              'Health (2011 Census);Bad health;',
              'Health (2011 Census);Very bad health;']
age_health = pd.read_csv(url,
                         usecols = ['Middle Super Output Area', 'MSOA Name',
                                    'Age Structure (2011 Census);All Ages;'] + age_cols + health_cols, 
                         low_memory = False, encoding = 'latin')

# Drop row on average for London and City of London
age_health.dropna(axis = 0, inplace = True)
age_health.drop(age_health[age_health['MSOA Name'] == 'City of London 001'].index, inplace = True)
age_health.reset_index(inplace = True)

# Change data type
age_health[age_cols] = age_health[age_cols].astype('int', copy = False)
age_health[health_cols] = age_health[health_cols].astype('int', copy = False)

# Drop numbers (last 4 characters) from MSOA name
for i in range(0, age_health.shape[0]):
               age_health.iloc[i,2] = age_health.iloc[i,2][:-4]

# Compute total number of respondents in MSOA
respondents = age_health.groupby('MSOA Name').agg({'Age Structure (2011 Census);All Ages;': np.sum}).reset_index()

# Compute number and percentage of respondents in each age group
age = age_health.groupby('MSOA Name').agg(dict.fromkeys(age_cols, np.sum)).reset_index()
age[age_cols] = age[age_cols].div(respondents['Age Structure (2011 Census);All Ages;'], axis = 0) * 100

# Compute number and percentage of respondents in each health category
health = age_health.groupby('MSOA Name').agg(dict.fromkeys(health_cols, np.sum)).reset_index()
health[health_cols] = health[health_cols].div(respondents['Age Structure (2011 Census);All Ages;'], axis = 0) * 100

# Rename columns for easy reference
age.rename({age_cols[0]: 'Age_0-15',
           age_cols[1]: 'Age_15-29',
           age_cols[2]: 'Age_30-44',
           age_cols[3]: 'Age_45-64',
           age_cols[4]: 'Age_65+',
           'MSOA Name': 'Area'},
          axis = 1, inplace = True)
health.rename({health_cols[0]: 'Health_Very_good',
               health_cols[1]: 'Health_Good',
               health_cols[2]: 'Health_Fair',
               health_cols[3]: 'Health_Bad',
               health_cols[4]: 'Health_Very_bad',
              'MSOA Name': 'Area'},
              axis = 1, inplace = True)

#### Economic Activity

In [8]:
# Read economic inactivity and rename columns for easy reference
# Source: https://data.london.gov.uk/dataset/economic-inactivity-gender
url = 'https://github.com/ngliangwei15/CASA0007-CW3/blob/main/Data/economic-inactivity.csv?raw=true'
econ_cols = ['Code', 'Area', 'percent; Jan 2011-Dec 2011']
economic = pd.read_csv(url, usecols = econ_cols, low_memory = False, encoding = 'latin')
economic.rename({econ_cols[2]: 'Economic_Inactive'}, axis = 1, inplace = True)

# Keep only data on London boroughs (Area Code starts with E09), drop City of London and change data type
economic.dropna(axis = 0, inplace = True)
economic = economic[economic['Code'].str.match(r'E09')].sort_values(by = ['Code'], ascending=True)
economic.drop(economic[economic.Area == 'City of London'].index, inplace = True)
economic = economic.reset_index(drop = True)
economic.drop('Code', axis = 1, inplace = True)

# Change data type
economic.Economic_Inactive = economic.Economic_Inactive.astype('float', copy = False)

IncompleteRead: IncompleteRead(32159140 bytes read, 42311062 more expected)

In [ ]:
# Read unemployment rate
# Source: https://data.london.gov.uk/dataset/employment-occupation-type-and-gender-borough
url = 'https://github.com/ngliangwei15/CASA0007-CW3/blob/main/Data/mb-unemployment-rates.xlsx?raw=true'
unemployment_cols = ['Code', 'Area', 'Jan 2011-Dec 2011']
unemployment = pd.read_excel(url, usecols = unemployment_cols, sheet_name = 'Rates')

# Keep only data on London boroughs (Area Code starts with E09), drop City of London and change data type
unemployment.dropna(axis = 0, inplace = True)
unemployment = unemployment[unemployment['Code'].str.match(r'E09')].sort_values(by = ['Code'], ascending=True)
unemployment.drop(unemployment[unemployment.Area == 'City of London'].index, inplace = True)
unemployment = unemployment.reset_index(drop = True)

# Change data type
unemployment['Jan 2011-Dec 2011'] = unemployment['Jan 2011-Dec 2011'].astype('float', copy = False)

In [ ]:
# Compute unemployment rate among economically active
economic['Economic_Unemployed'] = ((100 - economic.Economic_Inactive)/100) * (unemployment['Jan 2011-Dec 2011']/100) * 100

# Compute employment rate among economically active
economic['Economic_Employed'] = 100 - economic.Economic_Inactive - economic.Economic_Unemployed

### Descriptive Statistics

In [ ]:
# Create dictionary of variable names
dataset = {'dependent': ['wellbeing'], 
           'independent': ['ndvi'], 
           'mediators': ['anxiety', 'exercise', 'air_quality'], 
           'control': ['health', 'economic', 'age']}

# Initialize dataframe
data_descriptive = pd.DataFrame(globals()[dataset[[*dataset][0]][0]].Area)

# Merge data
for value in dataset.values():
    for var in value:
        data_descriptive = data_descriptive.merge(globals()[var], on = 'Area')
        
# Descriptive statistics
data_descriptive.describe()

## Step 1: Establish the effect of the independent variable on the dependent variables 

### Step 1a: Linear regressions of dependent variables on the independent variable

In [ ]:
# Merge data
data = pd.merge(wellbeing, ndvi, on = 'Area')
ind_var = ndvi.columns.tolist()[1:]

# Perform regression
for cat in wellbeing_cols:
    print(f'PERFORMING REGRESSION ON:{cat.upper()}')

    # Perform regression
    reg_model = sm.OLS(endog=data[cat], exog=sm.add_constant(data[ind_var])).fit()
    print(reg_model.summary())

    # plot residual vs. fit
    plt.scatter(reg_model.fittedvalues, reg_model.resid)
    plt.xlabel('Fitted Value')
    plt.ylabel('Residual')
    plt.title('Residual vs. Fitted Plot for ' + cat.title())
    plt.show()
    print('\n')

Urban greenery had positive effect on all three aspects of well-being, but were significant for LS and WW only.

### Step 1(b): Control variables were added to the regressions as additional independent variables.

In [ ]:
# Merge data
data = pd.merge(wellbeing, ndvi, on = 'Area')
data = data.merge(health, on = 'Area')
data = data.merge(economic, on = 'Area')
data = data.merge(age, on = 'Area')

In [ ]:
# Visualize collinearity
df = data.drop(['Area'] + wellbeing.columns.tolist(), axis = 1)
plt.figure(figsize=(16, 8))
cm = sns.diverging_palette(20, 220, as_cmap=True)
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, cmap = cm, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

#### (i) Using VIF to handle multi-collinearity

In [ ]:
# Define function to drop variables using VIF
# This function is adjusted from: https://stackoverflow.com/a/51329496/4667568

def drop_column_using_vif_(df, thresh=5):
    '''
    Calculates VIF each feature in a pandas dataframe, and repeatedly drop the columns with the highest VIF
    A constant must be added to variance_inflation_factor or the results will be incorrect

    :param df: the pandas dataframe containing only the predictor features, not the response variable
    :param thresh: (default 5) the threshould VIF value. If the VIF of a variable is greater than thresh, it should be removed from the dataframe
    :return: dataframe with multicollinear features removed
    '''
    while True:
        # adding a constant item to the data
        df_with_const = add_constant(df)

        vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) 
               for i in range(df_with_const.shape[1])], name= "VIF",
              index=df_with_const.columns).to_frame()

        # drop the const
        vif_df = vif_df.drop('const')
        
        # if the largest VIF is above the thresh, remove a variable with the largest VIF
        # If there are multiple variabels with VIF>thresh, only one of them is removed. This is because we want to keep as many variables as possible
        if vif_df.VIF.max() > thresh:
            # If there are multiple variables with the maximum VIF, choose the first one
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {}'.format(index_to_drop))
            df = df.drop(columns = index_to_drop)
        else:
            # No VIF is above threshold. Exit the loop
            break

    return df

In [ ]:
# Drop collinear variables
ind_var_data = drop_column_using_vif_(df)

# Variable names of remaining variables
ind_var = ind_var_data.columns.tolist()

In [ ]:
# Perform regression
for cat in wellbeing_cols:
    print(f'PERFORMING REGRESSION ON:{cat.upper()}')

    # Perform regression
    reg_model = sm.OLS(endog=data[cat], exog=sm.add_constant(data[ind_var])).fit()
    print(reg_model.summary())

    # plot residual vs. fit
    plt.scatter(reg_model.fittedvalues, reg_model.resid)
    plt.xlabel('Fitted Value')
    plt.ylabel('Residual')
    plt.title('Residual vs. Fitted Plot for ' + cat.title())
    plt.show()
    print('\n')

Urban greenery no longer had a significant effect on well-being, when other variables are controlled for.

#### (i) Using PCA to handle collinearity

In [ ]:
# Merge data of independent and control variables
data_pca = pd.merge(ndvi, health, on = 'Area')
data_pca = data_pca.merge(economic, on = 'Area')
data_pca = data_pca.merge(age, on = 'Area')

# Standardize data
cols = data_pca.columns.tolist()[1:]
scalers = [StandardScaler().fit(data_pca[x].values.reshape(-1,1)) for x in cols]
data_tr = data_pca.copy()
for i in range(0, len(cols)):
    data_tr.loc[:,cols[i]] = scalers[i].transform(data_tr[cols[i]].values.reshape(-1,1))

In [ ]:
# Perform PCA
pca = PCA(n_components=10, whiten=True) 
pca.fit(data_tr[cols])

# Check the variance
explained_variance = pca.explained_variance_ratio_
singular_values = pca.singular_values_

# Plot variance
x = np.arange(1,len(explained_variance)+1)
plt.plot(x, explained_variance)
plt.ylabel('Share of Variance Explained')
plt.show()

for i in range(0, 10):
    print(f"Component {i:>2} accounts for {explained_variance[i]*100:>2.2f}% of variance")

In [ ]:
# The first four components explain 93.31% of the variance. Hence, we will proceed with 4 components 

keep_n_components = 4
pca = PCA(n_components=keep_n_components, whiten=True)

data_tr_pca = pca.fit_transform(data_tr[cols])

In [ ]:
# Merge back with dataframe

for x in [data_tr_pca]:
    new_columns = []
    
    for i in range(0,keep_n_components):
        new_columns.append([])

    for i in x:
        for j in range(0,keep_n_components):
            new_columns[j].append(i[j])

    for i in range(0,keep_n_components):
        data_pca[f"Component_{i+1}"] = new_columns[i]

In [ ]:
# Eigenvectors
eig_vec = pca.components_
eig_vec_table = pd.DataFrame(cols, columns = ['Indepdendent Variable'])
eig_vec_table = eig_vec_table.join(pd.DataFrame(eig_vec).transpose())
eig_vec_table = eig_vec_table.rename({0: 'PC1', 1: 'PC2', 2: 'PC3', 3: 'PC4'}, axis = 1)

cm = sns.diverging_palette(20, 220, as_cmap=True)
x=eig_vec_table.style.background_gradient(cmap=cm)
display(x)

Urban greenery was highly loaded in PC1.

In [ ]:
# Prepare data for regression
data_pca.drop(['Health_Very_good', 'Health_Good', 'Health_Fair', 'Health_Bad', 'Health_Very_bad', 
               'Age_0-15', 'Age_15-29', 'Age_30-44', 'Age_45-64', 'Age_65+', 
               'Economic_Inactive', 'Economic_Unemployed', 'Economic_Employed', 
               'percentage_greenery'], axis = 1, inplace = True)

# Merge data
data_reg = wellbeing.merge(data_pca, on = 'Area')

In [ ]:
# Perform regression
for cat in wellbeing_cols:
    print(f'PERFORMING REGRESSION OF {cat.upper()}')
    
    # Perform regression
    reg_model = sm.OLS(endog=data_reg[cat], exog=sm.add_constant(data_reg[data_reg.columns.tolist()[4:]])).fit()
    #reg_model = back_regress(data_reout[cat], data_reout[group])
    print(reg_model.summary())

    # plot residual vs. fit
    plt.scatter(reg_model.fittedvalues, reg_model.resid)
    plt.xlabel('Fitted Value')
    plt.ylabel('Residual')
    plt.title('Residual vs. Fitted Plot for ' + cat.title())
    plt.show()
    print('\n')

PC1, which was negatively related to urban greenery, had a negative relationship with all aspects of well-being, and this was significant for LS and WW, like the observation in step 1(a).

## Step 2: Establish the effect of urban greenery on the mediators

In [ ]:
# Initialize dataframe to track coefficient and standard error
path_a = pd.DataFrame({'Mediators': dataset['mediators'], 
                       'Coeff_a': [0] * len(dataset['mediators']),
                       'se_a': [0] * len(dataset['mediators'])})

In [ ]:
# Get exogeneous variable
exog = globals()[dataset['independent'][0]]

# Perform regression
for mediator in dataset['mediators']:
    print(f'PERFORMING REGRESSION OF {mediator.upper()}')

    # Get endogeneous variable
    endog = globals()[mediator]
    
    # Merge data
    data_reg = pd.merge(endog, exog, on = 'Area')
    
    # Perform regression
    reg_model = sm.OLS(endog= data_reg[endog.columns.tolist()[1:]], 
                       exog=sm.add_constant(data_reg[exog.columns.tolist()[1:]])).fit()
    print(reg_model.summary())

    # plot residual vs. fit
    plt.scatter(reg_model.fittedvalues, reg_model.resid)
    plt.xlabel('Fitted Value')
    plt.ylabel('Residual')
    plt.title('Residual vs. Fitted Plot for ' + cat.title())
    plt.show()
    print('\n')
    
    # Store results
    path_a.loc[path_a.Mediators == mediator, 'Coeff_a'] = reg_model.params[1]
    path_a.loc[path_a.Mediators == mediator, 'se_a'] = reg_model.bse[1]

Urban greenery had a positive and significant effect on air quality, but no significant effect on the other two mediators. Hence, testing for the effect of the mediators on well-being proceeded for enhancement of air quality only.

## Step 3: Establish the effect of the mediators, which were established to be significantly affected by urban greenery, on personal well-being 

In [ ]:
# Initialize dataframe to track coefficient and standard error
path_b = pd.DataFrame({'Aspect': wellbeing_cols, 
                       'Coeff_b': [0] * len(wellbeing_cols),
                       'se_b': [0] * len(wellbeing_cols)})

In [ ]:
# Merge data
data = pd.merge(wellbeing, ndvi, on = 'Area')
data = data.merge(air_quality, on = 'Area')
ind_var = data.columns.tolist()[4:]

# Perform regression
for cat in wellbeing_cols:
    print(f'PERFORMING REGRESSION ON:{cat.upper()}')

    # Perform regression
    reg_model = sm.OLS(endog=data[cat], exog=sm.add_constant(data[ind_var])).fit()
    print(reg_model.summary())

    # plot residual vs. fit
    plt.scatter(reg_model.fittedvalues, reg_model.resid)
    plt.xlabel('Fitted Value')
    plt.ylabel('Residual')
    plt.title('Residual vs. Fitted Plot for ' + cat.title())
    plt.show()
    print('\n')
    
    # Store results
    path_b.loc[path_b.Aspect == cat, 'Coeff_b'] = reg_model.params[2]
    path_b.loc[path_b.Aspect == cat, 'se_b'] = reg_model.bse[2]

#### Sobel test

In [ ]:
# Create dataframe
sobel = pd.DataFrame({'Aspect': wellbeing_cols, 
                       'Coeff_a': path_a.loc[path_a.Mediators == 'air_quality', 'Coeff_a'].tolist() * len(wellbeing_cols),
                       'se_a': path_a.loc[path_a.Mediators == 'air_quality', 'se_a'].tolist() * len(wellbeing_cols)})
sobel = sobel.merge(path_b, on = 'Aspect')

# Compute Z-statistics and p-value
sobel['Z-stat'] = (sobel.Coeff_a * sobel.Coeff_b) / (
    (((sobel.Coeff_a**2) * (sobel.se_b**2)) + ((sobel.Coeff_b**2) * (sobel.se_a**2))).apply(sqrt))
sobel['p-value'] = norm.cdf(sobel['Z-stat'])

# Display values
sobel

Enhancement of air quality was a significant mediator between urban greenery, and LS and HP. While urban greenery had a positive relationship on both air quality and well-being, there was a negative relationship between air quality and well-being.